In [1]:
## Primeramente se descarga la base de datos que será la de perros y gatos
#import h5py
import cv2
import numpy as np
import os
import zipfile
from matplotlib import image

files=zipfile.ZipFile('cats_and_dogs_small.zip','r')
files.extractall('')

x_dog=[]
x_cat=[]


In [2]:
from PIL import Image
x_size=150
y_size=150

for name in files.namelist():
    if '/dogs/' in name and '.jpg' in name:
        a=cv2.imread(name)
        a=cv2.resize(a,(x_size,y_size)) # Dimensión de la imagen
        img = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        #img2=img.resize(200,200) # Mobilenet (224,224,3)
        x_dog.append(img)
        
    elif '/cats/' in name and '.jpg' in name:
        a=cv2.imread(name)
        a=cv2.resize(a,(x_size,y_size)) # Dimensión de la imagen
        img = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
        x_cat.append(img)
print(len(x_dog),len(x_cat))
x_dog=np.stack(x_dog,axis=0)
x_cat=np.stack(x_cat,axis=0)

2000 2000


# One Hot

In [3]:
print(type(x_dog),x_dog.shape)
print(type(x_cat),x_cat.shape)
x_dog=x_dog.astype('float32')
x,y,z,w=x_dog.shape
y_dog=np.zeros((x,1),dtype=int)
x_cat=x_cat.astype('float32')
x,y,z,w=x_cat.shape
y_cat=np.ones((x,1),dtype=int)
#x_dog=(x_dog/127.5)-1#x_dog/=255
#x_cat=(x_cat/127.5)-1#x_cat/=255
## Conjunto combinado de perros y gatos
x_comb=np.vstack((x_dog,x_cat))
y_comb=np.vstack((y_dog,y_cat))
print(x_comb.ndim,x_comb.shape)
#print(y_dog)


### ONE HOT
from keras.utils import to_categorical
y_dog_oh=to_categorical(y_dog,y_dog.max()+2)
y_cat_oh=to_categorical(y_cat,y_cat.max()+1)
print(y_comb[3455])
y_comb_oh=to_categorical(y_comb,y_comb.max()+1)
print(y_comb_oh[3455])
print(type(y_comb_oh),y_comb_oh.shape)
#print(y_cat_oh.shape)
#print(y_dog_oh.shape)
#print(y_dog_oh)
#print(y_cat_oh)

<class 'numpy.ndarray'> (2000, 150, 150, 3)
<class 'numpy.ndarray'> (2000, 150, 150, 3)
4 (4000, 150, 150, 3)


C:\Users\aldoa\anaconda3\envs\env2\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[1]
[0. 1.]
<class 'numpy.ndarray'> (4000, 2)


# División de datos 60 20 20

In [4]:
xx,yy,ww,zz=x_cat.shape
x_train=np.vstack((x_cat[:int(xx*0.6),:,:,:],x_dog[:int(xx*0.6),:,:,:]))
y_train=np.vstack((y_cat[:int(xx*0.6),:],y_dog[:int(xx*0.6),:]))
x_val=np.vstack((x_cat[int(xx*0.6):int(xx*0.8),:,:,:],x_dog[int(xx*0.6):int(xx*0.8),:,:,:]))
y_val=np.vstack((y_cat[int(xx*0.6):int(xx*0.8),:],y_dog[int(xx*0.6):int(xx*0.8),:]))
x_test=np.vstack((x_cat[int(xx*0.8):,:,:,:],x_dog[int(xx*0.8):,:,:,:]))
y_test=np.vstack((y_cat[int(xx*0.8):,:],y_dog[int(xx*0.8):,:]))

print(x_test.min(),x_test.max())

0.0 255.0


# Autokeras Library

In [5]:
import autokeras as ak
clf=ak.ImageClassifier(overwrite=True,max_trials=2)
clf.fit(x_train,y_train,epochs=3)


Trial 2 Complete [00h 10m 48s]
val_loss: 0.0

Best val_loss So Far: 0.0
Total elapsed time: 00h 12m 39s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
75/75 [==============================] - 258s 3s/step - loss: 0.5856 - accuracy: 0.9517
Epoch 2/3
75/75 [==============================] - 251s 3s/step - loss: 1.4074 - accuracy: 0.8800
Epoch 3/3
75/75 [==============================] - 263s 4s/step - loss: 2.0641 - accuracy: 0.7742


INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


In [8]:
## predicciones 
pred=clf.predict(x_test)

25/25 [==============================] - 26s 1s/step


In [9]:

pred=np.argmax(pred,axis=1)

exactitud_test=0
for a in range(len(pred)):
    if pred[a]==y_test[a]:
        exactitud_test+=1
print('exactitud de la prueba= ',100*exactitud_test/len(pred),'%')

exactitud de la prueba=  50.0 %


In [16]:


## Se propone una red neuronal ResNet50
import tensorflow as tf
from keras.models import Model, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
"""
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint =  ModelCheckpoint('resnet50.h', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
earlystopper = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1, restore_best_weights = True)    
x_train_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_train))
x_val_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_val))
x_test_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_test))
model = ResNet50(input_shape=(x_size,y_size,3),weights=None,include_top=False) ## Colocar otro top
for i in range(165):
    model.layers[i].trainable=False
#model.summary()
sal=model.output
sal=Flatten()(sal)
sal=Dense(502,activation='selu')(sal)
sal=Dropout(0.26)(sal)
sal = Dense(256, activation='relu')(sal)
sal=Dense(100,activation='selu')(sal)
sal=Dense(50,activation='relu')(sal)
sal = Dense(2, activation='softmax')(sal)
#Se unen la CNN y el top
resnet50_custom=Model(inputs=model.input, outputs=sal)
resnet50_custom.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=["accuracy"])
"""

x_train_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_train))
x_val_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_val))
x_test_res=tf.keras.applications.resnet50.preprocess_input(np.copy(x_test))

def model_builder(hp):

    model = ResNet50(input_shape=(x_size,y_size,3),weights=None,include_top=False) ## Colocar otro top
    for i in range(165):
        model.layers[i].trainable=False    
    sal=model.output
    sal=Flatten()(sal)
    sal=Dense(502,activation='selu')(sal)
    sal=Dropout(0.26)(sal)
    sal = Dense(256, activation='relu')(sal)
    sal=Dense(100,activation='selu')(sal)
    sal=Dense(50,activation='relu')(sal)
    sal = Dense(2, activation='softmax')(sal)
    #Se unen la CNN y el top
    resnet50_custom=Model(inputs=model.input, outputs=sal)
    resnet50_custom.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=["accuracy"])
    return model

# Se importa la librería keras_tuner

posteriormente se aplica el tuner, se eligío el Hyperband pues tiene las siguientes caracteristicas:

El algoritmo de sintonización Hyperband utiliza la asignación de recursos adaptable y la detención anticipada para converger rápidamente en un modelo de alto rendimiento.

In [25]:
import keras_tuner as kt 

tuner=kt.Hyperband(model_builder,objective='val_accuracy',max_epochs=5,factor=2)

In [28]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[:]

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [29]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=5, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.